In [13]:
# get a sense of data
    # how many comments
    # what's the distribution of score
    # what's the distribution of length
    # do same thing with user (number, score, total length by user)
    # 
# get data into documents
# remove, stem, chunk

In [65]:
from pymongo import MongoClient
from dotenv import load_dotenv
load_dotenv()
import os

%load_ext autoreload
%autoreload 1
import sys
sys.path.append('../src')

from data.scrapeReddit import get_all
from data.mymongo import get_db
%aimport data.scrapeReddit
%aimport data.mymongo

db = get_db()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
db.submissions.find().count()

/Users/willyraedy/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


147337

In [68]:
db.comments.aggregate([
    {'$match': {'subreddit': 'democrats'}},
    {'$group': {'_id': '$link_id', 'comment_text': {'$addToSet': '$body'}}}
]).next()

KeyboardInterrupt: 

In [54]:
db.comments.find().limit(1).next()

{'_id': ObjectId('5e4d98c866585708db994332'),
 'all_awardings': [],
 'associated_award': None,
 'author': 'LuminoZero',
 'author_flair_background_color': None,
 'author_flair_css_class': 'newyork-flag',
 'author_flair_richtext': [{'a': ':flag-ny:',
   'e': 'emoji',
   'u': 'https://emoji.redditmedia.com/uzfj5qhxoxe11_t5_2cneq/flag-ny'},
  {'e': 'text', 't': ' New York'}],
 'author_flair_template_id': 'bd600206-8e72-11e6-bf04-0ee844677561',
 'author_flair_text': ':flag-ny: New York',
 'author_flair_text_color': 'dark',
 'author_flair_type': 'richtext',
 'author_fullname': 't2_f9llx',
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'body': "This was my hope for Warren.  She won't win any district flat out, but she'll be *viable* in every district.  That could seriously help her.",
 'collapsed_because_crowd_control': None,
 'created_utc': 1580781599,
 'gildings': {},
 'id': 'fghc8e0',
 'is_submitter': False,
 'link_id': 't3_eyh51w',
 'locked': False,
 'no_follo

In [21]:
db.submissions.find().count()

/Users/willyraedy/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


156065

In [84]:
duplicates = db.comments.aggregate([
    {'$match': {'subreddit': {'$ne': 'politics'}}},
    {'$group': {'_id': '$id', 'id': {'$sum': 1}, '_id_to_delete': {'$last': '$_id'}}},
    {'$match': {'id': {'$gt': 1}}}
], allowDiskUse=True)
print('done with getting dups')
for rec in duplicates:
    id_val = rec['_id_to_delete']
    db.comments.delete_one({'_id': id_val})

done with getting dups


In [85]:
duplicates = db.comments.aggregate([
    {'$match': {'subreddit': {'$ne': 'politics'}}},
    {'$group': {'_id': '$id', 'id': {'$sum': 1}, '_id_to_delete': {'$last': '$_id'}}},
    {'$match': {'id': {'$gt': 1}}}
], allowDiskUse=True)
len(list(duplicates))

0